In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from glob import glob
import os

In [2]:
rel_data_dir_full = './data/sharew8_all/'
rel_data_dir_easy = './data/sharew8_easy/'
outdir = './Project'

In [3]:
'''
Wave 8 Release 8.0.0
DOI: 10.6103/SHARE.w8.800
2022-02-10 00:53:53
'''
all_files = glob(f"{rel_data_dir_full}*.dta")

'''
easySHARE Release 8.0.0
DOI: 10.6103/SHARE.easy.800
2022-02-10 01:00:22
'''
easy_files = glob(f"{rel_data_dir_easy}*.dta")

'''
Wave 8 Accelerometer Epochs Release 8.0.0
DOI: 10.6103/SHARE.w8.800
2022-02-10 01:04:04
'''
acc_files = glob(f"{rel_data_dir_full}dta/**/*.dta", recursive=True)

Loading the datasets from SHARE 

In [4]:
df_easy = pd.read_stata(easy_files[0])
df_ph = pd.read_stata(os.path.join(rel_data_dir_full, 'sharew8_rel8-0-0_ph.dta'))
df_acc_hr = pd.read_stata(os.path.join(rel_data_dir_full, 'sharew8_rel8-0-0_gv_accelerometer_hour.dta'))
df_acc = pd.read_stata(os.path.join(rel_data_dir_full, 'sharew8_rel8-0-0_gv_accelerometer_total.dta'))
df_acc_imp = pd.read_stata(os.path.join(rel_data_dir_full, 'sharew8_rel8-0-0_gv_imputations.dta'))
df_acscore = pd.read_stata(os.path.join(rel_data_dir_full, 'sharew8_rel8-0-0_br.dta'))
df_acc_hr_clean = df_acc_hr.iloc[:,0:15]

Data prep

In [5]:
df_acc_hr_clean.loc[:, 'OxCGRT_stay_at_home'] = df_acc_hr_clean['OxCGRT_stay_at_home'].replace({
    'No measures': 'No restrictions',
    "Require not leaving house with exceptions for daily exercise, grocery shopping, and 'essential' trips": 'Essential only',
    "Recommend not leaving house": 'Stay at home',
})

In [6]:
def get_common_columns(df1, df2, merge_on=['mergeid']):
    common = df1.columns.difference(df2.columns).tolist()
    common.extend(merge_on)
    return common

In [7]:
df = df_acc_hr_clean.copy()
df = pd.merge(left=df, left_on=['mergeid'], right=df_ph[get_common_columns(df_ph, df)], right_on=['mergeid'], how='left')
df = pd.merge(left=df, left_on=['mergeid'], right=df_acc[get_common_columns(df_acc, df)], right_on=['mergeid'], how='left')
df = pd.merge(left=df, left_on=['mergeid'], right=df_easy[get_common_columns(df_easy, df)], right_on=['mergeid'], how='left')
df = pd.merge(left=df, left_on=['mergeid'], right=df_acscore[get_common_columns(df_acscore, df)], right_on=['mergeid'], how='left')
df = df[df['wave'] == 8]

In [31]:
def map_selected_num(df, columns: list[str]):
    df = df.copy()
    for c in columns:
        df.loc[:, c] = df[c].map({
            'Selected': 1,
            'Not selected': 0,
            "Don't know": -1,
            'Refusal': -2,
            1: 1,
            0: 0,
            -1: -1,
            -2: -2,
        })
    return df

def map_bmi_num(df, columns: list[str] = ['bmi']):
    df = df.copy()
    for c in columns:
        df.loc[:, c] = df[c].replace({
            '-15. no information': -1,
            '-13. not asked in this wave': -1,
            "-12. don't know / refusal": -1,
            '-3. implausible value/suspected wrong': -1,
        })
        df[c] = df[c].astype(float)
    return df

def map_freq_num(df, columns:list[str]):
    df = df.copy()
    for c in columns:
        df.loc[:, c] = df[c].replace({
            'More than once a week': 1,
            'Once a week': 2,
            'One to three times a month': 3,
            'Hardly ever, or never': 4,
            "Don't know": -1,
            'Refusal' : -1,
            '1. More than once a week': 1,
            '2. Once a week': 2,
            '3. One to three times a month': 3,
            '4. Hardly ever, or never': 4,
            "5. Don't know": -1,
            '6. Refusal' : -1,
        })
        df[c] = df[c].astype(float)
    return df

In [32]:
#                        Hypertension, Diabetis, Osteoarthritis, pain hips, pain knee
df = map_selected_num(df, ['ph006d2', 'ph006d5', 'ph006d20', 'ph087d2', 'ph087d3'])
df = map_bmi_num(df)
df = map_freq_num(df, ['br015_', 'br016_'])   

ValueError: Cannot cast object dtype to float64

In [17]:
df['female'] = pd.to_numeric(df['female'].replace({'1. female': 1, '0. male': 0}))
df.loc[:, 'age'] = df.replace({'-15. no information': -1})
df.loc[:, 'age'] = df['age'].astype(float)

Defining Osteoarthritis 

ph006d20: Osteoarthritis/other rheumatism: ever diagnosed/currently having

ph087d2: Pain location: hips or
ph087d3: Pain location: knees


d1: Hypertension

d2: Diabetes

d3: OA (in hip/knee)

d4: Hypertension + Diabetes

d5: Hypertension + OA

d6: Diabetes + OA

d7: Hypertension + Diabetes + OA  

In [18]:
df['Hypertension'] = pd.to_numeric(df['ph006d2'])
df['Diabetes'] = pd.to_numeric(df['ph006d5'])
df['OA'] = np.where(((df['ph006d20'] == 1) & (df['ph087d2'] == 1)) | ((df['ph006d20'] == 1) & (df['ph087d3'] == 1)), int(1), int(0))

In [19]:
df['d1'] = np.where((df['Hypertension'] == 1) & (df['Diabetes'] == 0) & (df['OA'] == 0), int(1), int(0))
df['d2'] = np.where((df['Hypertension'] == 0) & (df['Diabetes'] == 1) & (df['OA'] == 0), int(1), int(0))
df['d3'] = np.where((df['Hypertension'] == 0) & (df['Diabetes'] == 0) & (df['OA'] == 1), int(1), int(0))
df['d4'] = np.where((df['Hypertension'] == 1) & (df['Diabetes'] == 1) & (df['OA'] == 0), int(1), int(0))
df['d5'] = np.where((df['Hypertension'] == 1) & (df['Diabetes'] == 0) & (df['OA'] == 1), int(1), int(0))
df['d6'] = np.where((df['Hypertension'] == 0) & (df['Diabetes'] == 1) & (df['OA'] == 1), int(1), int(0))
df['d7'] = np.where((df['Hypertension'] == 1) & (df['Diabetes'] == 1) & (df['OA'] == 1), int(1), int(0))
df['control'] = np.where((df['Hypertension'] == 0) & (df['Diabetes'] == 0) & (df['OA'] == 0), int(1), int(0))

In [20]:
df['disease']  = df[['d1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'control',]].apply(lambda x: x.idxmax(), axis=1)
df['disease'].value_counts()

control    68352
d1         43728
d4         12336
d3          8616
d2          8040
d5          7944
d7          3648
d6           936
Name: disease, dtype: int64

Self-reported activity score

In [21]:
df['activity'] = 2*(4-df['br015_']) + (4-df['br016_'])

TypeError: unsupported operand type(s) for -: 'int' and 'Categorical'

In [21]:
df.drop(df[df.measurementday > 7].index, inplace=True)

In [24]:
vars = ['mergeid', 'female', 'age', 'bmi', 'activity', 'GGIR_mean_ENMO_hour','GGIR_N_valid_hours','hour','measurementday','weekday','month',
        'year','device_id', 'frequency', 'position', 'country', 'disease', 'd1','d2','d3', 'd4','d5','d6','d7','control']
df_stat = df[vars].copy()
df_stat.rename(columns={'GGIR_mean_ENMO_hour': 'ENMO_hour'}, inplace=True)

In [25]:
df_stat.head(5)

,mergeid,female,age,bmi,activity,ENMO_hour,GGIR_N_valid_hours,hour,measurementday,weekday,...,country,disease,d1,d2,d3,d4,d5,d6,d7,control
4,Bf-031257-01,0,67.0,21.155294,NaN,6.697222,0.0,12am-1am,1,Tuesday,...,Belgium,control,0,0,0,0,0,0,0,1
9,Bf-031257-01,0,67.0,21.155294,NaN,10.741667,0.0,1am-2am,1,Tuesday,...,Belgium,control,0,0,0,0,0,0,0,1
14,Bf-031257-01,0,67.0,21.155294,NaN,7.848472,0.5,2am-3am,1,Tuesday,...,Belgium,control,0,0,0,0,0,0,0,1
19,Bf-031257-01,0,67.0,21.155294,NaN,0.244306,1.0,3am-4am,1,Tuesday,...,Belgium,control,0,0,0,0,0,0,0,1
24,Bf-031257-01,0,67.0,21.155294,NaN,4.214167,1.0,4am-5am,1,Tuesday,...,Belgium,control,0,0,0,0,0,0,0,1


Export to csv

In [21]:
df_stat.to_csv(f"{outdir}/merged_data.csv", index = False)